In [2]:
%load_ext autoreload
%autoreload 2

import subprocess; FOLDER_PATH = subprocess.Popen(['git', 'rev-parse', '--show-toplevel'], stdout=subprocess.PIPE).communicate()[0].rstrip().decode('utf-8')
import warnings; warnings.filterwarnings("ignore")
import sys, os; sys.path.append(os.path.dirname(f'{FOLDER_PATH}/utils')); sys.path.append(os.path.dirname(f'{FOLDER_PATH}/scripts'))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [35]:
import optuna
from keras.backend import clear_session
import tensorflow as tf
from tensorflow.keras import models, layers

from utils.tf_helpers import WindowGenerator, compile_and_fit, plot_plantwise_predictions
from utils.base_helpers import read_data, split_data, scale_data, expand_data, plot_metrics
from utils.graph_helpers import generate_adj_matrix, create_graph, LSTMGC, calculate_wmape_tf as wmape


In [22]:
df, plants = read_data(generate_speed_angle=True, add_lagged=True, number_of_plants=8)
train_df_, valid_df, test_df = split_data(df, train_ratio=0.8, valid_ratio=0.1)
train_df, valid_df, test_df = expand_data(train_df_, valid_df, test_df)
train_df, valid_df, test_df = scale_data(train_df, valid_df, test_df, plants, scaler="minmax")

Train start and end dates:	 2019-01-26 03:00:00 	 2021-06-22 09:00:00
Validation start and end dates:	 2021-06-22 10:00:00 	 2021-10-10 04:00:00
Test start and end dates:	 2021-10-10 05:00:00 	 2022-01-27 23:00:00


In [23]:
OUT_STEPS = 24
INPUT_WIDTH = 30

window = WindowGenerator(
    train_df=train_df, valid_df=valid_df, test_df=test_df, 
    columns=[col for col in df.columns if col != "rt_plant_id"],
    input_width=INPUT_WIDTH, label_width=OUT_STEPS, shift=0, label_columns=["production"])

In [26]:
adjacency_matrix = generate_adj_matrix(train_df_, threshold=0.6)
graph = create_graph(adjacency_matrix)

number of nodes: 8, number of edges: 60


In [39]:
MAX_EPOCHS = 10
OUT_FEAT = 64
lstm_units = 32
BATCH_SIZE = 64

NUM_NODES = window.number_of_plants
IN_FEAT = len(window.feature_column_indices)
INPUT_SEQ_LEN = INPUT_WIDTH
forecast_horizon = OUT_STEPS
multi_horizon = True


def objective(trial):
    clear_session()
    
    graph_aggregation_type = trial.suggest_categorical("graph_aggregation_type", ["mean", "sum", "max"])
    graph_combination_type = trial.suggest_categorical("graph_combination_type", ["concat", "add"])
    graph_activation = trial.suggest_categorical("graph_activation", ["relu", "linear"])
    lstm_activation = trial.suggest_categorical("lstm_activation", ["sigmoid", "relu", "tanh"])
    dense_activation = trial.suggest_categorical("dense_activation", ["relu", "sigmoid"])
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-1, log=True)
    loss = trial.suggest_categorical("loss", ["mse", "binary_crossentropy"])
    
    graph_conv_params = {"aggregation_type": graph_aggregation_type, "combination_type": graph_combination_type, "activation": graph_activation,}
    keras_params = {"lstm_activation": lstm_activation, "dense_activation": dense_activation}

    st_gcn = LSTMGC(IN_FEAT, OUT_FEAT, lstm_units, INPUT_SEQ_LEN, 
        forecast_horizon, graph, graph_conv_params, keras_params)
    inputs = layers.Input((INPUT_SEQ_LEN, graph.num_nodes, IN_FEAT))
    outputs = st_gcn(inputs)

    model = models.Model(inputs, outputs)

    model, history = compile_and_fit(model, window, max_epochs=10, verbose=1, loss=loss,
                                     optimizer=tf.keras.optimizers.RMSprop(learning_rate=learning_rate))
    
    score = model.evaluate(window.test, verbose=1)
    return score[1]

In [37]:
# import wandb
# from wandb.keras import WandbCallback

# wandb.init(project="optuna", entity="merts")

wandb: Currently logged in as: merts (use `wandb login --relogin` to force relogin)
2022-04-26 23:08:00.496640: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-26 23:08:00.496657: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [38]:
from optuna.integration.wandb import WeightsAndBiasesCallback

wandbc = WeightsAndBiasesCallback(metric_name="accuracy", wandb_kwargs={"project": "optuna"})

2022-04-26 23:10:34.150751: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-26 23:10:34.150767: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [ ]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=100, timeout=1200, callbacks=[wandbc])

print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2022-04-26 23:10:53,765] A new study created in memory with name: no-name-ac06254b-d5e6-4f71-a2b9-6f4fd30dc0fd


Epoch 1/10
329/329 [==============================] - ETA: 0s - loss: 0.1341 - wmape: 0.9125
Epoch 1: val_wmape improved from inf to 0.68141, saving model to /home/mert/Desktop/thesis/artifacts/checkpoint
329/329 [==============================] - 11s 30ms/step - loss: 0.1341 - wmape: 0.9125 - val_loss: 0.1151 - val_wmape: 0.6814
Epoch 2/10
328/329 [============================>.] - ETA: 0s - loss: 0.1253 - wmape: 0.8818
Epoch 2: val_wmape improved from 0.68141 to 0.66774, saving model to /home/mert/Desktop/thesis/artifacts/checkpoint
329/329 [==============================] - 9s 27ms/step - loss: 0.1253 - wmape: 0.8817 - val_loss: 0.1106 - val_wmape: 0.6677
Epoch 3/10
328/329 [============================>.] - ETA: 0s - loss: 0.1192 - wmape: 0.8587
Epoch 3: val_wmape improved from 0.66774 to 0.65373, saving model to /home/mert/Desktop/thesis/artifacts/checkpoint
329/329 [==============================] - 9s 27ms/step - loss: 0.1192 - wmape: 0.8584 - val_loss: 0.1063 - val_wmape: 0.653

[I 2022-04-26 23:12:26,751] Trial 0 finished with value: 0.6545836925506592 and parameters: {'graph_aggregation_type': 'sum', 'graph_combination_type': 'add', 'graph_activation': 'linear', 'lstm_activation': 'sigmoid', 'dense_activation': 'sigmoid', 'learning_rate': 1.5855167918220696e-05, 'loss': 'mse'}. Best is trial 0 with value: 0.6545836925506592.


Epoch 1/10
328/329 [============================>.] - ETA: 0s - loss: 0.8827 - wmape: 0.5794
Epoch 1: val_wmape improved from inf to 0.51243, saving model to /home/mert/Desktop/thesis/artifacts/checkpoint
329/329 [==============================] - 12s 33ms/step - loss: 0.8824 - wmape: 0.5791 - val_loss: 0.8528 - val_wmape: 0.5124
Epoch 2/10
329/329 [==============================] - ETA: 0s - loss: 0.7023 - wmape: 0.4732
Epoch 2: val_wmape improved from 0.51243 to 0.43157, saving model to /home/mert/Desktop/thesis/artifacts/checkpoint
329/329 [==============================] - 11s 33ms/step - loss: 0.7023 - wmape: 0.4732 - val_loss: 0.7841 - val_wmape: 0.4316
Epoch 3/10
329/329 [==============================] - ETA: 0s - loss: 0.6914 - wmape: 0.4461
Epoch 3: val_wmape improved from 0.43157 to 0.40877, saving model to /home/mert/Desktop/thesis/artifacts/checkpoint
329/329 [==============================] - 11s 32ms/step - loss: 0.6914 - wmape: 0.4461 - val_loss: 0.7964 - val_wmape: 0.4

[I 2022-04-26 23:14:16,722] Trial 1 finished with value: 0.384509414434433 and parameters: {'graph_aggregation_type': 'sum', 'graph_combination_type': 'concat', 'graph_activation': 'linear', 'lstm_activation': 'sigmoid', 'dense_activation': 'relu', 'learning_rate': 0.0020698589074901455, 'loss': 'binary_crossentropy'}. Best is trial 1 with value: 0.384509414434433.


...

Epoch 1/10
329/329 [==============================] - ETA: 0s - loss: 0.5428 - wmape: 0.5920
Epoch 1: val_wmape improved from inf to 0.37124, saving model to /home/mert/Desktop/thesis/artifacts/checkpoint
329/329 [==============================] - 14s 37ms/step - loss: 0.5428 - wmape: 0.5920 - val_loss: 0.5266 - val_wmape: 0.3712
Epoch 2/10
328/329 [============================>.] - ETA: 0s - loss: 0.4669 - wmape: 0.4025
Epoch 2: val_wmape improved from 0.37124 to 0.33321, saving model to /home/mert/Desktop/thesis/artifacts/checkpoint
329/329 [==============================] - 12s 36ms/step - loss: 0.4668 - wmape: 0.4025 - val_loss: 0.5088 - val_wmape: 0.3332
Epoch 3/10
329/329 [==============================] - ETA: 0s - loss: 0.4559 - wmape: 0.3722
Epoch 3: val_wmape improved from 0.33321 to 0.31777, saving model to /home/mert/Desktop/thesis/artifacts/checkpoint
329/329 [==============================] - 12s 35ms/step - loss: 0.4559 - wmape: 0.3722 - val_loss: 0.5018 - val_wmape: 0.3

[I 2022-04-26 23:25:10,730] Trial 6 finished with value: 0.3150860369205475 and parameters: {'graph_aggregation_type': 'mean', 'graph_combination_type': 'add', 'graph_activation': 'relu', 'lstm_activation': 'tanh', 'dense_activation': 'sigmoid', 'learning_rate': 0.00030924598310278926, 'loss': 'binary_crossentropy'}. Best is trial 5 with value: 0.30958807468414307.


Epoch 1/10
328/329 [============================>.] - ETA: 0s - loss: 1.8780 - wmape: 1.1921
Epoch 1: val_wmape improved from inf to 0.76526, saving model to /home/mert/Desktop/thesis/artifacts/checkpoint
329/329 [==============================] - 13s 36ms/step - loss: 1.8735 - wmape: 1.1908 - val_loss: 0.2001 - val_wmape: 0.7653
Epoch 2/10
329/329 [==============================] - ETA: 0s - loss: 0.1500 - wmape: 0.7386
Epoch 2: val_wmape improved from 0.76526 to 0.66935, saving model to /home/mert/Desktop/thesis/artifacts/checkpoint
329/329 [==============================] - 12s 35ms/step - loss: 0.1500 - wmape: 0.7386 - val_loss: 0.1637 - val_wmape: 0.6693
Epoch 3/10
329/329 [==============================] - ETA: 0s - loss: 0.1257 - wmape: 0.6540
Epoch 3: val_wmape improved from 0.66935 to 0.58712, saving model to /home/mert/Desktop/thesis/artifacts/checkpoint
329/329 [==============================] - 12s 35ms/step - loss: 0.1257 - wmape: 0.6540 - val_loss: 0.1348 - val_wmape: 0.5

[I 2022-04-26 23:27:11,845] Trial 7 finished with value: 0.34086930751800537 and parameters: {'graph_aggregation_type': 'sum', 'graph_combination_type': 'add', 'graph_activation': 'relu', 'lstm_activation': 'relu', 'dense_activation': 'relu', 'learning_rate': 0.008021477749997303, 'loss': 'mse'}. Best is trial 5 with value: 0.30958807468414307.


Epoch 1/10
329/329 [==============================] - ETA: 0s - loss: 0.0670 - wmape: 0.5070
Epoch 1: val_wmape improved from inf to 0.39114, saving model to /home/mert/Desktop/thesis/artifacts/checkpoint
329/329 [==============================] - 16s 44ms/step - loss: 0.0670 - wmape: 0.5070 - val_loss: 0.0568 - val_wmape: 0.3911
Epoch 2/10
329/329 [==============================] - ETA: 0s - loss: 0.0378 - wmape: 0.3899
Epoch 2: val_wmape improved from 0.39114 to 0.33294, saving model to /home/mert/Desktop/thesis/artifacts/checkpoint
329/329 [==============================] - 12s 35ms/step - loss: 0.0378 - wmape: 0.3899 - val_loss: 0.0390 - val_wmape: 0.3329
Epoch 3/10
329/329 [==============================] - ETA: 0s - loss: 0.0326 - wmape: 0.3613
Epoch 3: val_wmape improved from 0.33294 to 0.32081, saving model to /home/mert/Desktop/thesis/artifacts/checkpoint
329/329 [==============================] - 11s 34ms/step - loss: 0.0326 - wmape: 0.3613 - val_loss: 0.0328 - val_wmape: 0.3